In [33]:
# %pip install -U lightgbm==3.3.2

In [34]:
# %pip install implicit

In [35]:

# !pip3 uninstall -y numpy gensim
# !pip3 install numpy==1.26.4 gensim==4.3.3 --no-cache-dir


In [36]:
# pip list | grep -E 'numpy|gensim|scipy|pandas'


In [37]:
import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import gc
from pathlib import Path

from gensim.models import Word2Vec

import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
sys.path.append("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/") # path to the `src`` folder
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()

In [38]:
from src.data import DataHelper
import pickle

In [39]:
from collections import defaultdict

In [40]:
data_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/Vibrent")
model_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/models/models_Vibrent")
# data_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/HM")
# model_dir = Path("/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/models/models_HM")

In [41]:
# !pip3 install fastparquet


In [42]:
dh = DataHelper(data_dir)
data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved
data = dh.load_data(name="encoded_full")

Encode Item Sparse Feats: 100%|██████████| 12/12 [00:00<00:00, 180.31it/s]


### Train


In [43]:
inter = data['inter']

In [44]:
inter['t_dat'] = pd.to_datetime(inter['t_dat'])
# last_week_start = pd.to_datetime("2020-08-19")
last_week_start = pd.to_datetime("2023-10-30")
inter = inter.loc[(inter.t_dat < last_week_start)]

In [45]:
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).values

In [46]:
# model_sg = Word2Vec(feedid_seq_list,  size=128, window=32, min_count=1, sg=0, sample=1e-3, negative=15, workers=32, seed=1, iter=10)
# model_sg.save(open(model_dir/'articleid_model_cbow.model','wb'))
model_sg = Word2Vec(feedid_seq_list,  
                     vector_size=128,  # Đổi 'size' thành 'vector_size'
                     window=32, 
                     min_count=1, 
                     sg=1, 
                     sample=1e-3, 
                     negative=15, 
                     workers=32, 
                     seed=1, 
                     epochs=10)  # Đổi 'iter' thành 'epochs'

# Lưu mô hình đúng cách
model_sg.save(str(model_dir / 'articleid_model_skipgram.model'))


In [47]:
# model_sg = pickle.load(open(model_dir/'articleid_model_cbow.model','rb'))
model_sg = Word2Vec.load(str(model_dir / 'articleid_model_skipgram.model'))


In [48]:

# !pip3 install gensim==3.8.3


In [49]:
# * Customer Embedding
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).reset_index()

full_users = data['user']['customer_id'].values
customer_embedding = np.ones((len(full_users)+1, 128))/128

for uid, items in tqdm(feedid_seq_list.values):
    if len(items) > 1:
        vec = np.mean(
    [model_sg.wv[str(x)] for x in items if str(x) in model_sg.wv.key_to_index], axis=0)
    else:
        vec = model_sg.wv[str(items[0])] if str(items[0]) in model_sg.wv.key_to_index else np.zeros(model_sg.vector_size)
    customer_embedding[uid] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 2214/2214 [00:00<00:00, 21067.44it/s]


In [50]:
del full_users, feedid_seq_list, inter
gc.collect()

0

In [51]:
# The line `# customer_embedding.dump(data_dir/'external'/'w2v_user_embd.npy')` is a comment in the code. Comments are used to provide explanations or notes within the code for better understanding by developers. In this case, it seems like the code is intended to dump or save the customer embedding data to a file named 'w2v_user_embd.npy' in the 'external' directory within the specified data directory. However, the line is commented out, so it is not currently active or executed when the code runs.
customer_embedding.dump(data_dir/'external'/'w2v_user_embd.npy')
customer_embedding.dump(data_dir/'external'/'w2v_skipgram_user_embd.npy')

In [52]:
# * Article Embedding
full_items = data['item']['article_id'].values
article_embedding = np.ones((len(full_items)+1, 128))/128
for item in tqdm(full_items):
    try:
        vec = model_sg[str(item)] 
    except:
        vec = article_embedding[item]
    article_embedding[item,:] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 15649/15649 [00:00<00:00, 210536.55it/s]


In [53]:
article_embedding.dump(data_dir/'external'/'w2v_item_embd.npy')
article_embedding.dump(data_dir/'external'/'w2v_skipgram_item_embd.npy')

In [54]:
# * Product_code Embedding
full_products = list(data['item']['product_code'].unique())

In [55]:
product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = article_embedding[items['article_id'].values]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

100%|██████████| 9756/9756 [00:00<00:00, 43255.79it/s]


In [56]:
product_embedding = np.ones((len(full_products)+1, 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid,:] = embd

100%|██████████| 9756/9756 [00:00<00:00, 527865.81it/s]


In [57]:
product_embedding.dump(data_dir/'external'/'w2v_product_embd.npy')
product_embedding.dump(data_dir/'external'/'w2v_skipgram_product_embd.npy')

---

In [58]:
# * Product_code Embedding
full_products = list(data['item']['product_code'].unique())

In [59]:
dssm_item_embd = np.load(data_dir/'external'/'dssm_item_embd.npy', allow_pickle=True)

product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = dssm_item_embd[items['article_id'].values-1]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

product_embedding = np.ones((len(full_products), 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid-1,:] = embd

100%|██████████| 9756/9756 [00:00<00:00, 1715779.69it/s]


In [60]:
product_embedding.dump(data_dir/'external'/'dssm_product_embd.npy')

In [61]:
yt_item_embd = np.load(data_dir/'external'/'yt_item_embd.npy', allow_pickle=True)

product_embd_dict = {}
for pid, items in tqdm(data['item'].groupby('product_code')):
    embd = yt_item_embd[items['article_id'].values-1]
    embd = np.sum(embd, axis=0)
    product_embd_dict[pid] = embd/np.sqrt(np.sum(embd**2))

product_embedding = np.ones((len(full_products), 128))/128
for pid,embd in tqdm(product_embd_dict.items()):
    product_embedding[pid-1,:] = embd

100%|██████████| 9756/9756 [00:00<00:00, 1639933.87it/s]


In [62]:
product_embedding.dump(data_dir/'external'/'yt_product_embd.npy')

In [63]:
np.save(data_dir/'external'/'dssm_user_embd.npy', customer_embedding)

In [64]:
# yt_interactions = pd.read_csv('/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/Vibrent/raw/transactions_train.csv')
yt_interactions = pd.read_csv('/Users/tuanthanh/Documents/KLTN/SourceClone/H-M-Fashion-RecSys/data/Vibrent/raw/transactions_train.csv')
yt_feedid_seq_list = yt_interactions.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x]).reset_index()

# Initialize the embedding array for yt_users
full_yt_users = yt_interactions['customer_id'].unique()
yt_user_embedding = np.ones((len(full_yt_users) + 1, 128)) / 128

user_id_to_index = {user_id: idx for idx, user_id in enumerate(full_yt_users)}

for uid, items in tqdm(yt_feedid_seq_list.values):
    user_index = user_id_to_index.get(uid) 
    if user_index is None:
        continue 

    if len(items) > 1:
        vec = np.mean([model_sg.wv[str(x)] for x in items if str(x) in model_sg.wv.key_to_index], axis=0)
    else:
        vec = model_sg.wv[str(items[0])] if str(items[0]) in model_sg.wv.key_to_index else np.zeros(model_sg.vector_size)

    # Normalize the embedding
    yt_user_embedding[user_index] = vec / np.sqrt(np.sum(vec**2))

del full_yt_users, yt_feedid_seq_list, yt_interactions
gc.collect()

np.save(data_dir / 'external' / 'yt_user_embd.npy', yt_user_embedding)


100%|██████████| 2293/2293 [00:00<00:00, 75152.88it/s]


0